#### Importación de módulos

In [1]:
import pyspark.sql.functions as f_
from pyspark.sql.functions import current_timestamp
from itertools import chain
from pyspark.sql.window import Window
import sys
sys.path.append('/lakehouse/default/Files/GENERALES')
#%run "/lakehouse/default/Files/GENERALES/f_amr.py"
import f_amr

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 3, Finished, Available, Finished)

- ##### Construcción de Funnel

###### Selección de columnas iniciales

In [2]:
bdd_corte_pbi_unitec = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/BBDD_CORTE_PBI_UNITEC.parquet')
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
bdd_corte_pbi_unitec = f_amr.cast_cols(bdd_corte_pbi_unitec, {'UNICOS_ACTUAL':'int','UNICOS_PY':'int',"INSCRITOS_BRUTOS":'int', "TOTALES_ACTUAL":'int'})
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.withColumn("UNICOS_ACTUAL",f_.when((f_.col("VARIABLE") == "INSCRITOS") & (f_.col("INSCRITOS_BRUTOS") == 1), f_.col("TOTALES_ACTUAL")).otherwise(f_.col("UNICOS_ACTUAL")))
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.filter(f_.col('UNICOS_ACTUAL') == 1)
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.withColumn('RECICLADO', f_.when(f_.trim(f_.col('TIPO_RECICLADO')).startswith('R'), 'RECICLADO').otherwise('NUEVO'))
seleccion_4 = \
{'CAMPUS':'CAMPUS',
'CARRERA':'CARRERA',
'CICLO':'CICLO',
'CICLO_LAUREATE':'CICLO_LAUREATE',
'FECHA':'FECHA',
'FUENTE':'FUENTE',
'FUENTE_AGRUPADA':'FUENTE_AGRUPADA',
'LINEA_NEGOCIO_ING':'LINEA_NEGOCIO_ING',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'RECICLADO':'RECICLADO',
'TIPO_DE_PROGRAMA':'TIPO_DE_PROGRAMA',
'TIPO_DE_INGRESO':'TIPO_DE_INGRESO',
'TIPO_TMK':'TEAM_TMK',
'UNICOS_ACTUAL':'UNICOS_ACTUAL',
'UNICOS_PY':'UNICOS_PY',
'UTILIDAD':'UTILIDAD',
'VARIABLE':'ETAPA'}
bdd_corte_pbi_unitec = f_amr.select_cols(bdd_corte_pbi_unitec, seleccion_4)
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'FECHA')))
bdd_corte_pbi_unitec=bdd_corte_pbi_unitec.withColumn("FECHA", f_.to_date("FECHA", "yyyy-MM-dd"))

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 4, Finished, Available, Finished)

###### Adición y corrección de columnas básicas KPI

- Catálogos

In [3]:
# Fuente
funnel_unitec_a = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/FUNNEL_UNITEC_A.parquet')
funnel_unitec_a = funnel_unitec_a.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
# Limpieza de columnas
seleccion_2 = \
{'CARRERA':'CARRERA',
'CICLO':'CICLO',
'FECHA':'FECHA',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'AGRUPADOR_SITIO':'AGRUPADOR_SITIO',
'LINEA_H':'LINEA_H'}

funnel_unitec_a = f_amr.select_cols(funnel_unitec_a, seleccion_2)
funnel_unitec_a = f_amr.uniques(funnel_unitec_a, ['NO_DE_PERSONA', 'CICLO'], [('NO_DE_PERSONA', 'asc'),('FECHA', 'desc')])
cat_carrera = funnel_unitec_a.select('CARRERA', 'LINEA_H').distinct()

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 5, Finished, Available, Finished)

- Ajuste columnas KPI

In [4]:
# Adición selectiva
match_Agrcl_1 = bdd_corte_pbi_unitec.drop('CARRERA').join(funnel_unitec_a.select('NO_DE_PERSONA', 'CICLO', 'CARRERA', 'AGRUPADOR_SITIO', 'LINEA_H'),['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_Agrcl_left_1 = bdd_corte_pbi_unitec.join(funnel_unitec_a,['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_Agrcl_right_1 = funnel_unitec_a.join(bdd_corte_pbi_unitec,['NO_DE_PERSONA','CICLO'], 'left_anti')

match_Agrcl_2 = no_match_Agrcl_left_1.drop('CARRERA').join(no_match_Agrcl_right_1.select('NO_DE_PERSONA', 'CARRERA', 'AGRUPADOR_SITIO', 'LINEA_H'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])
no_match_Agrcl_left_2 = no_match_Agrcl_left_1.join(no_match_Agrcl_right_1, 'NO_DE_PERSONA', 'left_anti')
match_Agrcl_3 = no_match_Agrcl_left_2.join(cat_carrera, 'CARRERA', 'left').dropDuplicates(['INDICE'])

bdd_corte_pbi_unitec = match_Agrcl_1.unionByName(match_Agrcl_2, allowMissingColumns=True).unionByName(match_Agrcl_3, allowMissingColumns=True)

# Normalización post uniones
bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.withColumn('AGRUPADOR_SITIO', f_.when(f_.col('AGRUPADOR_SITIO').isNull(), 'NO ES MARKETING').otherwise(f_.col('AGRUPADOR_SITIO')))

# Adición de Semana
calendario = spark.read.option('header', True).option('inferSchema', True).csv('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/CALENDARIO.csv')
calendario = calendario.withColumn('FECHA', f_.to_date('FECHA', 'dd/MM/yyyy'))
calendario = f_amr.cast_cols(calendario, {'SEMANA':'int'})

bdd_corte_pbi_unitec = bdd_corte_pbi_unitec.join(calendario.select('FECHA', 'SEMANA'), on='FECHA', how='inner').drop('INDICE')

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 6, Finished, Available, Finished)

- Etapas

In [5]:
curva_segmentos = spark.read.parquet('abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_SEGMENTOS')

# Curva segmentos agrupada
seleccion_5 = \
['BLOQUES',
 'CABECERA',
 'CALL_SCORE',
 'CARRERA_EGRESO',
 'CICLO',
 'CITAS_CCENTER',
 'ESTATUS_MARCACION',
 'FUENTE_OBTENCION_CRM',
 'general_risk_bi',
 'general_risk_crm',
 'general_risk_prueba',
 'general_risk_unicos',
 'LEAD_SCORE',
 'LISTA',
 'NO_DE_PERSONA',
 'SCORE_UNICOS',
 'SEGMENTO_AGRUPADO_CCENTER',
 'SEGMENTO_CCENTER',
 'SEGMENTO_DESGLOSE',
 'SEGMENTO_GENERAL',
 'SEGMENTO_VIGENTE',
 'SEMANA',
 'TELEFONO_PREDICTIVO'
]
curva_segmentos_agrupada = curva_segmentos.select(seleccion_5).withColumnRenamed('SEMANA', 'SEMANA_MARCACION')
curva_segmentos_agrupada = curva_segmentos_agrupada.distinct()

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 7, Finished, Available, Finished)

In [6]:
# Contacto efectivo
ce = bdd_corte_pbi_unitec.filter(f_.col('ETAPA') == 'CONTACTO EFECTIVO').withColumn('CE', f_.lit(1))
ce = ce.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_ce_curvseg_1 = ce.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_ce_curvseg_1_left = ce.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_ce_curvseg_1_right = curva_segmentos_agrupada.join(ce, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_ce_curvseg_2 = no_match_ce_curvseg_1_left.join(no_match_ce_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

ce = match_ce_curvseg_1.unionByName(match_ce_curvseg_2, allowMissingColumns=True)

# Registros
leads = bdd_corte_pbi_unitec.filter(f_.col('ETAPA') == 'LEADS')
leads = leads.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_leads_curvseg_1 = leads.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_leads_curvseg_1_left = leads.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_leads_curvseg_1_right = curva_segmentos_agrupada.join(leads, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_leads_curvseg_2 = no_match_leads_curvseg_1_left.join(no_match_leads_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

leads = match_leads_curvseg_1.unionByName(match_leads_curvseg_2, allowMissingColumns=True)

# Citas
citas = bdd_corte_pbi_unitec.filter(f_.col('ETAPA') == 'CITAS')
citas = citas.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_citas_curvseg_1 = citas.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_citas_curvseg_1_left = citas.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_citas_curvseg_1_right = curva_segmentos_agrupada.join(citas, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_citas_curvseg_2 = no_match_citas_curvseg_1_left.join(no_match_citas_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

citas = match_citas_curvseg_1.unionByName(match_citas_curvseg_2, allowMissingColumns=True)

# Asistencias
asistencias = bdd_corte_pbi_unitec.filter(f_.col('ETAPA') == 'ASISTENCIAS')
asistencias = asistencias.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_asistencias_curvseg_1 = asistencias.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_asistencias_curvseg_1_left = asistencias.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_asistencias_curvseg_1_right = curva_segmentos_agrupada.join(asistencias, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_asistencias_curvseg_2 = no_match_asistencias_curvseg_1_left.join(no_match_asistencias_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

asistencias = match_asistencias_curvseg_1.unionByName(match_asistencias_curvseg_2, allowMissingColumns=True)

# Inscritos
inscritos = bdd_corte_pbi_unitec.filter(f_.col('ETAPA') == 'INSCRITOS')
inscritos = inscritos.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_inscritos_curvseg_1 = inscritos.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_inscritos_curvseg_1_left = inscritos.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_inscritos_curvseg_1_right = curva_segmentos_agrupada.join(inscritos, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_inscritos_curvseg_2 = no_match_inscritos_curvseg_1_left.join(no_match_inscritos_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

inscritos = match_inscritos_curvseg_1.unionByName(match_inscritos_curvseg_2, allowMissingColumns=True)

#Funnel
apilado_funnel = ce.unionByName(leads, allowMissingColumns=True)\
                   .unionByName(citas, allowMissingColumns=True)\
                   .unionByName(asistencias, allowMissingColumns=True)\
                   .unionByName(inscritos, allowMissingColumns=True)
apilado_funnel = apilado_funnel.withColumn('DB_DISPONIBLE', f_.lit(0))

curva_intentos_score = curva_segmentos.unionByName(apilado_funnel, allowMissingColumns=True).drop(*['INDICE', 'CICLO_LAUREATE'])

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 8, Finished, Available, Finished)

- Normalización de columnas

In [7]:
# Creación de columnas de identificación y corrección de filtros
ajustes_cols1 = \
{
 'Id_Persona_Ciclo': f_.concat_ws( '', 'NO_DE_PERSONA', 'CICLO'),
 'FECHA_CORTE': f_.expr("current_timestamp() - INTERVAL 1 DAY"),
 'CORTE': f_.when(f_.col("FECHA") <= f_.col("FECHA_CORTE"), "CORTE AL DIA").otherwise(""),
 'SEM_EQ': f_.when(f_.col("SEGMENTO_AGRUPADO_CCENTER").contains(f_.concat(f_.lit("S"), f_.col("SEMANA").cast("string"))), "SI").otherwise("NO"),
 'SEM_CONV': f_.when(f_.col('SEMANA_MARCACION').isNull(), 'SI').when(f_.col('SEMANA_MARCACION') == f_.col('SEMANA'), 'SI').otherwise('NO')
} 

curva_intentos_score = curva_intentos_score.withColumns(ajustes_cols1)

c = {'25-2': 'C1','25-3': 'C2','26-1': 'C3'}
curva_intentos_score = f_amr.map_cols(curva_intentos_score,'CICLO',c,'CICLO_LAUREATE')

# Dummies por etapa
etapas = ['LEADS', 'CITAS', 'ASISTENCIAS', 'INSCRITOS']
for i in etapas:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col('ETAPA') == i,f_.lit(1)).otherwise(f_.lit(0)) )

#Completitud de scores nulos ,
sin_score_1 = ['general_risk_bi', 'general_risk_crm', 'general_risk_prueba', 'general_risk_unicos']
for i in sin_score_1:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(i).isNull(), "S/SCORE").otherwise(f_.col(i)))

sin_score_bin = [('S/SCORE_bi','general_risk_bi'),('S/SCORE_crm', 'general_risk_crm'),( 'S/SCORE_prueba','general_risk_prueba'),('S/SCORE_unicos','general_risk_unicos')]
for (i, j) in sin_score_bin:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j) == "S/SCORE",1).otherwise(0))

#Completitud de scores nulos
sin_score_2 = [('S/SCORE_LS','LEAD_SCORE'), ('S/SCORE_CS','CALL_SCORE'), ('S/SCORE_UNICOS_H','SCORE_UNICOS')]
for (i,j) in sin_score_2:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j).isNull(), 1).otherwise(0))

# Generación de intervalos
intervalos = [('INTERVALO_SC_bi', 'S/SCORE_LS', 'LEAD_SCORE'),
('INTERVALO_SC_crm', 'S/SCORE_CS', 'CALL_SCORE'),
('INTERVALO_SC_unicos', 'S/SCORE_UNICOS_H', 'SCORE_UNICOS')]
for (i,j,k) in intervalos:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j)==1, 'S/SCORE')\
                                                                .when(f_.col(k)>= 90, '[90,100]')\
                                                                .when(f_.col(k)>= 80, '[80,90)')\
                                                                .when(f_.col(k)>= 70, '[70,80)')\
                                                                .when(f_.col(k)>= 60, '[60,70)')\
                                                                .when(f_.col(k)>= 50, '[50,60)')\
                                                                .when(f_.col(k)>= 40, '[40,50)')\
                                                                .when(f_.col(k)>= 30, '[30,40)')\
                                                                .when(f_.col(k)>= 20, '[20,30)')\
                                                                .when(f_.col(k)>= 10, '[10,20)')\
                                                                .otherwise('[0,10)'))

#identificadores score
id_score = {"GREEN":'1',"YELLOW":'2',"ORANGE":'3',"RED":'4',"S/SCORE":"5"}
ids_score = [('general_risk_bi', 'ID_General_Risk_bi'),
('general_risk_crm', 'ID_General_Risk_crm'),
('general_risk_prueba','ID_General_Risk_prueba'),
('general_risk_unicos', 'ID_General_Risk_unicos')]
for (i,j) in ids_score:
    curva_intentos_score = f_amr.map_cols(curva_intentos_score, i , id_score, j)

id_rango = {"[90,100]":'10', "[80,90)":'9', "[70,80)":'8', "[60,70)":'7' ,"[50,60)":'6' ,"[40,50)":'5' ,"[30,40)":'4',"[20,30)":'3',"[10,20)":'2', "[0,10)":'1',"S/SCORE":'11'}
ids_rango = [('INTERVALO_SC_bi', 'ORDEN_INTERVALO_bi'),
('INTERVALO_SC_crm', 'ORDEN_INTERVALO_crm'),
('INTERVALO_SC_unicos','ORDEN_INTERVALO_unicos')]
for (i,j) in ids_rango:
    curva_intentos_score = f_amr.map_cols(curva_intentos_score, i , id_rango, j)


StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 9, Finished, Available, Finished)

In [8]:
from pyspark.sql.functions import current_timestamp
curva_intentos_score = curva_intentos_score.withColumn("ACTUALIZACION", f_.expr("current_timestamp() - INTERVAL 6 HOURS"))
#curva_intentos_score.repartition(20).write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("CURVAS.CURVA_INTENTOS_SCORE")
p1= 'abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/CUBOS/CURVA_INTENTOS'
curva_intentos_score.repartition(10).write.mode("overwrite").parquet(p1)

StatementMeta(, d3ca94b5-cef0-4ec0-a4e9-66389deaf70a, 10, Submitted, Running, Running)